In [2]:
from dotenv import load_dotenv
import os
import openai 
import pandas as pd 

# Carregar variáveis do arquivo .env
load_dotenv()

# Verificar se a chave foi carregada
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

# Configurar o cliente OpenAI com a chave carregada
openai.api_key = api_key

In [3]:
client = openai.Client()

In [4]:
vector_store = client.beta.vector_stores.create(
    name="Tutor de Apostilas"
)

In [5]:
file = ["files/LLM.pdf"]

In [6]:
file_stream = [open(f,"rb") for f in file]
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files = file_stream
)

In [7]:
file_batch.status

'completed'

In [8]:
file_batch.file_counts

FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)

In [9]:
assistant = client.beta.assistants.create(
    name="Tutor Apostila",
    instructions="Voce é um tutor especializado em tecnologias emergentes. \
        Voce sabe responder perguntas sobre LLMs como OpenAi, HuggingFace, etc. \
        Caso voce não encontre as respostas, seja sincero e fale que não sabe responder",
    tools=[{"type":"file_search"}],
    tool_resources={"file_search":{"vector_store_ids":[vector_store.id]}},
    model="gpt-4-turbo-preview"
)

In [10]:
pergunta = "Conforme o documento, o que é HuggingFace?"

In [11]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

In [12]:
# Executa a Thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome do usuário premium"
)

In [13]:
# Aguarda a thread rodar
import time 
while run.status in ["queued","in_progress","cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [14]:
# Verificar a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")

SyncCursorPage[Message](data=[Message(id='msg_mgXb1a4Ht9JtZKf4XY8fqEou', assistant_id='asst_6oSa67orzxifGGrbteKKP4x6', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=1222, file_citation=FileCitation(file_id='file-VAnVp1aS1itthEsKKDYkLa'), start_index=1209, text='【4:0†LLM.pdf】', type='file_citation')], value='HuggingFace é descrita como uma comunidade de código aberto que reúne centenas de milhares de modelos contribuídos por diversos colaboradores. Esses modelos podem ajudar a resolver uma ampla gama de casos de uso específicos, como geração de texto, resumo e classificação. A comunidade de código aberto, incluindo a HuggingFace, está avançando rapidamente para alcançar o desempenho de modelos proprietários, embora ainda não tenha conseguido igualar o desempenho de modelos de ponta como o GPT-4. Os modelos da HuggingFace e outros modelos de código aberto geralmente requerem mais esforço para serem implementados, mas 

In [15]:
mensagens.data[0].content[0].text.value

'HuggingFace é descrita como uma comunidade de código aberto que reúne centenas de milhares de modelos contribuídos por diversos colaboradores. Esses modelos podem ajudar a resolver uma ampla gama de casos de uso específicos, como geração de texto, resumo e classificação. A comunidade de código aberto, incluindo a HuggingFace, está avançando rapidamente para alcançar o desempenho de modelos proprietários, embora ainda não tenha conseguido igualar o desempenho de modelos de ponta como o GPT-4. Os modelos da HuggingFace e outros modelos de código aberto geralmente requerem mais esforço para serem implementados, mas progressos significativos estão sendo feitos para torná-los mais acessíveis aos usuários. Além disso, destacou-se a facilidade com que alguém com um pouco de experiência em Python pode pegar qualquer modelo transformador da HuggingFace e usá-lo como um objeto Python, destacando a flexibilidade e a possibilidade de manter os dados sob controle do usuário para preocupações com p

In [16]:
# Analisando os passos do modelo

run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [20]:
for step in run_steps.data[::-1]:
    print(F"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" * 10)
            # Verifique se o tool_call tem o atributo `code_interpreter`
            if hasattr(tool_call, "code_interpreter"):
                print(tool_call.code_interpreter.input)
            else:
                print(f"ToolCall não possui 'code_interpreter': {tool_call}")
    print("=" * 10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        print(message.content[0].text.value)


====Step tool_calls
ToolCall não possui 'code_interpreter': FileSearchToolCall(id='call_V760ftqVWZ6Iil5xU8vQDqMY', file_search=FileSearch(ranking_options=FileSearchRankingOptions(ranker='default_2024_08_21', score_threshold=0.0), results=[FileSearchResult(file_id='file-VAnVp1aS1itthEsKKDYkLa', file_name='LLM.pdf', score=0.513213399695081, content=None), FileSearchResult(file_id='file-VAnVp1aS1itthEsKKDYkLa', file_name='LLM.pdf', score=0.5006500892253628, content=None), FileSearchResult(file_id='file-VAnVp1aS1itthEsKKDYkLa', file_name='LLM.pdf', score=0.31152131604407635, content=None), FileSearchResult(file_id='file-VAnVp1aS1itthEsKKDYkLa', file_name='LLM.pdf', score=0.28506882404131934, content=None), FileSearchResult(file_id='file-VAnVp1aS1itthEsKKDYkLa', file_name='LLM.pdf', score=0.2848309003869383, content=None), FileSearchResult(file_id='file-VAnVp1aS1itthEsKKDYkLa', file_name='LLM.pdf', score=0.243150085390779, content=None), FileSearchResult(file_id='file-VAnVp1aS1itthEsKKDYkL